In [1]:
import os
from os import path

import numpy as np
import torch
from torch import optim
from torchvision import transforms
import pickle

from knockoff import datasets
import knockoff.config as cfg
from knockoff.adversary.policies import RandomAdversary
from knockoff.victim.blackbox import Blackbox
from knockoff.adversary.train import samples_to_transferset, get_optimizer
from knockoff.models import zoo
import knockoff.utils.model as model_utils


In [2]:
# hyper parameters
device = torch.device("cuda")

np.random.seed(cfg.DEFAULT_SEED)
torch.manual_seed(cfg.DEFAULT_SEED)
torch.cuda.manual_seed(cfg.DEFAULT_SEED)

transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
queryset = datasets.ImageNet1k(
    train=True,
    transform=transform,
)

testset = datasets.ImageNet1k(train=False, transform=transform)

out_dir = "./models/adversary/ADV_DIR"
os.makedirs(out_dir, exist_ok=True)
batch_size = 8
nworkers = 10
budget = 80000

policy = "random"
blackbox = Blackbox.from_modeldir("./models/victim/cubs200-resnet34")


=> done loading ImageNet1k (train) with 1024934 examples
=> done loading ImageNet1k (test) with 256233 examples
=> loading checkpoint './models/victim/cubs200-resnet34/checkpoint.pth.tar'
=> loaded checkpoint (epoch 96, acc=77.11)


/home/proj/anaconda3/envs/knockoff/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [13]:
# get transferset
adversary = RandomAdversary(blackbox, queryset, batch_size=batch_size)
transferset = adversary.get_transferset(budget)

# or load it
# with open(path.join(out_dir, "transferset.pickle"), "rb") as rf:
#     transferset = pickle.load(rf)

# pickle transfer
with open(path.join(out_dir, 'transferset.pickle'), 'wb') as wf:
    pickle.dump(transferset, wf)


RuntimeError: CUDA error: device-side assert triggered

In [12]:
[i[1] for i in testset.samples if i[1] == 999]

[999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999,
 999

In [4]:
num_classes = transferset[0][1].size(0)
budgets = [60000]

model = zoo.get_net('resnet34', 'imagenet', pretrained='imagenet', num_classes=num_classes).to(device)

for b in budgets:
    t = samples_to_transferset(transferset, budget=b, transform=transform)
    optimizer = get_optimizer(model.parameters(), 'sgdm')
    criterion_train = model_utils.soft_cross_entropy
    model_utils.train_model(model, t, out_dir, testset=testset, criterion_train=criterion_train,
                                checkpoint_suffix=f'.{b}', device=device, optimizer=optimizer)


/home/proj/anaconda3/envs/knockoff/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[Train] Epoch: 0.11 [6336/60000 (11%)]	Loss: 4.823625	Accuracy: 5.4 (347/6400)
[Train] Epoch: 0.21 [12736/60000 (21%)]	Loss: 4.479277	Accuracy: 8.2 (1047/12800)
[Train] Epoch: 0.32 [19136/60000 (32%)]	Loss: 4.147687	Accuracy: 11.0 (2110/19200)
[Train] Epoch: 0.43 [25536/60000 (43%)]	Loss: 4.144309	Accuracy: 13.1 (3354/25600)
[Train] Epoch: 0.53 [31936/60000 (53%)]	Loss: 4.137110	Accuracy: 14.7 (4710/32000)
[Train] Epoch: 0.64 [38336/60000 (64%)]	Loss: 4.004222	Accuracy: 16.0 (6159/38400)
[Train] Epoch: 0.75 [44736/60000 (75%)]	Loss: 3.827962	Accuracy: 17.3 (7746/44800)
[Train] Epoch: 0.85 [51136/60000 (85%)]	Loss: 3.895128	Accuracy: 18.4 (9438/51200)
[Train] Epoch: 0.96 [57536/60000 (96%)]	Loss: 3.724599	Accuracy: 19.4 (11160/57600)


/home/proj/anaconda3/envs/knockoff/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/conda-bld/pytorch_1616554793803/work/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1616554793803/work/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/con

RuntimeError: CUDA error: device-side assert triggered